In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import os, sys
from pathlib import Path


def find_src_folder():
    current = Path(os.getcwd()).resolve()
    for p in [current] + list(current.parents):
        src = p / "src"
        if src.exists():
            return src
    raise RuntimeError("src 폴더를 찾을 수 없습니다.")


src_path = find_src_folder()
sys.path.append(str(src_path))

In [ ]:
# %%writefile dungeon_event_agent.py
from langchain.chat_models import init_chat_model
from enums.LLM import LLM
from agents.dungeon.dungeon_state import DungeonEventParser
import random

llm = init_chat_model(model=LLM.GPT5_1, temperature=0.5)

from prompts.promptmanager import PromptManager
from prompts.prompt_type.dungeon.DungeonPromptType import DungeonPromptType
from agents.dungeon.dungeon_state import DungeonEventState
from db.RDBRepository import RDBRepository
from langchain_core.messages import HumanMessage


def heroine_memories_node(state: DungeonEventState) -> DungeonEventState:
    from agents.dungeon.event.heroine_scenarios import HEROINE_SCENARIOS

    heroine_id = state["heroine_data"]["heroine_id"]
    memory_progress = state["heroine_data"]["memory_progress"]

    # 해당 히로인의 해금된 기억들을 필터링함
    heroine_memories = [
        scenario
        for scenario in HEROINE_SCENARIOS
        if scenario["heroine_id"] == heroine_id
        and scenario["memory_progress"] <= memory_progress
    ]

    print(f"[heroine_memories_node] 히로인 ID: {heroine_id}")
    print(f"[heroine_memories_node] 기억 진척도: {memory_progress}")
    print(f"[heroine_memories_node] 해금된 기억 개수: {len(heroine_memories)}")

    return {"heroine_memories": heroine_memories}


def selected_main_event_node(state: DungeonEventState) -> DungeonEventState:
    from agents.dungeon.event.main_event_scenarios import MAIN_EVENT_SCENARIOS

    next_floor = state.get("next_floor", 1)
    used_events = state.get("used_events", [])

    # 이미 사용한 event_code 추출
    used_event_codes = [
        evt.get("event_code") for evt in used_events if "event_code" in evt
    ]

    # 사용 가능한 이벤트 필터링 (중복 제외)
    available_events = []
    for event in MAIN_EVENT_SCENARIOS:
        # 이미 사용한 이벤트 제외
        if event["event_code"] in used_event_codes:
            continue

        available_events.append(event)

    # 사용 가능한 이벤트가 없으면 모든 이벤트 풀에서 선택 (중복 허용)
    if not available_events:
        available_events = MAIN_EVENT_SCENARIOS

    # 랜덤 선택
    selected_event = random.choice(available_events)

    # 선택된 이벤트의 전체 정보를 문자열로 포맷팅
    event_description = f"""[{selected_event['title']}]
Event Code: {selected_event['event_code']}
Type: {'개별 이벤트' if selected_event['is_personal'] else '공통 이벤트'}

{selected_event['scenario_text']}"""

    print(f"[selected_main_event_node] 선택된 이벤트: {selected_event['title']}")
    print(f"[selected_main_event_node] 개별 이벤트 여부: {selected_event['is_personal']}")

    return {"selected_main_event": event_description}


def create_sub_event_node(state: DungeonEventState) -> DungeonEventState:
    prompts = PromptManager(DungeonPromptType.DUNGEON_SUB_EVENT).get_prompt(
        heroine_data=state["heroine_data"],
        heroine_memories=state["heroine_memories"],
        selected_main_event=state["selected_main_event"],
        event_room=state["event_room"],
        next_floor=state["next_floor"],
    )

    parser_llm = llm.with_structured_output(DungeonEventParser)
    response = parser_llm.invoke(prompts)

    # 서브 이벤트 결과 포맷팅 (EventChoice 객체를 문자열로 변환)
    choices_text = "\n".join(
        [f"{i+1}. {choice.action}" for i, choice in enumerate(response.event_choices)]
    )

    sub_event_result = f"""
=== 서브 이벤트 내러티브 ===
{response.sub_event_narrative}

=== 선택지 ===
{choices_text}

=== 예상 결과 ===
{response.expected_outcome}
"""

    final_answer = (
        f"=== 메인 이벤트 ===\n{state['selected_main_event']}\n\n"
        f"=== 서브 이벤트 (히로인 특화) ===\n{sub_event_result}"
    )

    print(f"[create_sub_event_node] 서브 이벤트 생성 완료")
    print(response)

    return {
        "messages": [HumanMessage(sub_event_result)],
        "sub_event": sub_event_result,
        "final_answer": final_answer,
    }


from langgraph.graph import START, END, StateGraph


graph_builder = StateGraph(DungeonEventState)
graph_builder.add_node("heroine_memories_node", heroine_memories_node)
graph_builder.add_node("selected_main_event_node", selected_main_event_node)
graph_builder.add_node("create_sub_event_node", create_sub_event_node)

graph_builder.add_edge(START, "heroine_memories_node")
graph_builder.add_edge("heroine_memories_node", "selected_main_event_node")

graph_builder.add_edge("selected_main_event_node", "create_sub_event_node")
graph_builder.add_edge("create_sub_event_node", END)
graph_builder

In [4]:
# from agents.dungeon.dungeon_event_agent import graph_builder
graph_builder.compile()

ValueError: Failed to reach https://mermaid.ink API while trying to render your graph. Status code: 400.

To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`

In [22]:
result = graph_builder.compile().invoke(
    {
        "heroine_data": {
            "heroine_id": 1,
            "memory_progress": 80,
        },
        "event_room": 3,
        "next_floor": 2,
    }
)
print("\n" + "=" * 50)
print(result["final_answer"])

[heroine_memories_node] 히로인 ID: 1
[heroine_memories_node] 기억 진척도: 80
[heroine_memories_node] 해금된 기억 개수: 5
[selected_main_event_node] 선택된 이벤트: 제단속에 고여있는 물
[selected_main_event_node] 개별 이벤트 여부: False
[create_sub_event_node] 서브 이벤트 생성 완료
sub_event_narrative='방의 중앙, 매끈한 검은 석재로 된 사각 기둥이 서 있다. 당신의 가슴께까지 올라오는 높이의 기둥 상단은 네모난 그릇처럼 깊게 파여 있고, 그 안에 물이 고여 있다. 이 물은 이 층의 눅눅한 공기와는 다르게 맑고 잔잔하며, 바람 한 점 없는데도 표면에 미세한 물결이 번져 간헐적으로 벽과 천장의 잔광을 일그러뜨린다. 기둥 주위 바닥은 건조해, 어디서 이 물이 공급되는지 알 수 없다. 가까이 다가가자, 물속에서 잠깐 당신의 윤곽이 비쳤다가, 낯선 실루엣으로 뒤틀리며 사라진다.' event_choices=[EventChoice(action='기둥에 다가가 물을 손으로 떠 마신다', reward_id='hp_recovery_all', penalty_id=None), EventChoice(action='손을 담가 물의 성질을 느끼며 잠시 집중한다', reward_id='cooldown_reduction_all', penalty_id='slow_debuff'), EventChoice(action='물속에 자신의 얼굴을 비춰보며 한동안 응시한다', reward_id='crit_chance_increase_all', penalty_id='curse_debuff'), EventChoice(action='기둥과 물을 건드리지 않고 주변만 살핀 뒤 방을 떠난다', reward_id=None, penalty_id=None)] expected_outcome='1번 선택지는 물을 직접 마셔 체력을 100만큼 회복(hp_recovery_

In [13]:
# 특정 이벤트(심연을 숭배하는 자)로 테스트
from agents.dungeon.event.main_event_scenarios import MAIN_EVENT_SCENARIOS

# ABYSS_WORSHIPPER 이벤트를 직접 선택
abyss_event = next(
    (e for e in MAIN_EVENT_SCENARIOS if e["event_code"] == "ABYSS_WORSHIPPER"), None
)

if abyss_event:
    event_description = f"""[{abyss_event['title']}]
Event Code: {abyss_event['event_code']}
Type: {'개별 이벤트' if abyss_event['is_personal'] else '공통 이벤트'}

{abyss_event['scenario_text']}"""

    print("=== 테스트할 메인 이벤트 ===")
    print(event_description)
    print("\n" + "=" * 50 + "\n")

    # 직접 create_sub_event_node만 테스트
    from prompts.promptmanager import PromptManager
    from prompts.prompt_type.dungeon.DungeonPromptType import DungeonPromptType

    prompts = PromptManager(DungeonPromptType.DUNGEON_SUB_EVENT).get_prompt(
        heroine_data={"heroine_id": 1, "memory_progress": 30},
        heroine_memories=[],
        selected_main_event=event_description,
        event_room=3,
        next_floor=2,
    )

    parser_llm = llm.with_structured_output(DungeonEventParser)
    response = parser_llm.invoke(prompts)

    print("=== 서브 이벤트 결과 ===")
    print(f"내러티브: {response.sub_event_narrative}\n")
    print("선택지:")
    for i, choice in enumerate(response.event_choices, 1):
        print(f"{i}. {choice.action}")
        if choice.reward_id:
            print(f"   보상: {choice.reward_id}")
        if choice.penalty_id:
            print(f"   패널티: {choice.penalty_id}")
    print(f"\n예상 결과: {response.expected_outcome}")

=== 테스트할 메인 이벤트 ===
[심연을 숭배하는 자]
Event Code: ABYSS_WORSHIPPER
Type: 공통 이벤트

방 한가운데에 후드를 쓴 인물이 등장한다. 인물에게 말을 걸면 엄청 더듬으며, 전혀 이해할 수 없는 이상한 말만 내뱉는다.


=== 서브 이벤트 결과 ===
내러티브: 방 안은 축축한 냉기와 함께 희미한 메아리로 가득하다. 중앙에는 메인 이벤트에서 보았던 그 후드를 깊게 눌러쓴 인물이 그대로 서 있다. 낡은 천은 물먹은 듯 어둡게 젖어 있고, 끝자락에서 물방울이 바닥으로 떨어지며 규칙적인 소리를 낸다. 얼굴은 여전히 그림자에 묻혀 보이지 않지만, 인물의 어깨가 불규칙하게 떨리고 있어 숨소리가 거칠게 새어 나오는 것이 느껴진다. 당신이 한 발 다가서자, 그는 머리를 약간 틀며 의미를 알 수 없는 말들을 끊긴 숨 사이로 토해낸다. 단어들은 이곳 레테의 언어와도, 어떤 기도와도 맞지 않는 기괴한 울림으로, 기억을 긁어내는 듯 머릿속을 불편하게 자극한다.

선택지:
1. 후드를 쓴 인물에게 천천히 말을 걸며, 그가 내뱉는 이상한 말을 따라 해 본다
   보상: skill_damage_increase_all
2. 후드를 쓴 인물을 일정 거리에서 조용히 관찰하며, 그의 몸짓과 말의 패턴을 분석한다
   보상: crit_chance_increase_all
3. 불길함을 느끼고 인물을 강하게 밀쳐 떼어놓으려 한다
   패널티: instant_damage_medium
4. 그를 완전히 무시하고, 옆을 스쳐 지나가며 방을 빠르게 벗어난다
   보상: move_speed_increase_all
   패널티: curse_debuff

예상 결과: 1번 선택지는 인물이 내뱉는 기억의 파편 같은 언어에 스스로를 맞추는 행동으로, 잠시 정신이 뒤틀리는 감각 끝에 스킬에 대한 이해가 날카로워져 스킬 데미지 +20% 버프를 얻는다.
2번 선택지는 직접 개입하지 않고 패턴을 읽어내는 방식으로, 인물이 움직일 때의 미세한 틈을 깨닫게 되어 치명타 확률 +10

In [20]:
# PromptManager 디버깅
from prompts.promptmanager import PromptManager
from prompts.prompt_type.dungeon.DungeonPromptType import DungeonPromptType

pm = PromptManager(DungeonPromptType.DUNGEON_SUB_EVENT)
template = pm.get_template()

print(f"Template name: {template.name}")
print(f"Input variables: {template.input_variables}")
print(
    f"\nPrompt에 {{selected_main_event}}가 있나? {'{selected_main_event}' in template.prompt}"
)

# 직접 format 시도
test_data = {
    "heroine_data": {"test": 1},
    "heroine_memories": [],
    "selected_main_event": "테스트 이벤트 내용",
    "event_room": 3,
    "next_floor": 2,
}

try:
    formatted = template.prompt.format(**test_data)
    if "{selected_main_event}" in formatted:
        print("\n❌ format() 후에도 변수가 남아있음!")
    else:
        print("\n✅ format() 성공!")
        # 메인 이벤트 부분만 확인
        start = formatted.find("<메인 이벤트>")
        end = formatted.find("</메인 이벤트>") + len("</메인 이벤트>")
        print(formatted[start:end])
except Exception as e:
    print(f"\n❌ format() 오류: {e}")

Template name: dungeon_sub_event
Input variables: ['heroine_data', 'heroine_memories', 'selected_main_event', 'event_room', 'next_floor']

Prompt에 {selected_main_event}가 있나? True

✅ format() 성공!
<메인 이벤트>
테스트 이벤트 내용
</메인 이벤트>
